In [1]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import ray
from ray import tune
from ray.tune.schedulers import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import ConfigSpace as CS
from functools import partial

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump, load

In [13]:
# Device configuration
device = torch.device('cuda')
classes = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
model_type = 'FFNN_Flat'

In [8]:
def plot_loss(data_train, data_val):
    plt.clf()
    plt.figure()
    plt.rcParams["figure.figsize"] = (13,13)
    plt.rcParams["legend.fontsize"] = 12
    plt.xlabel('Epoch',fontsize=16)
    plt.ylabel('Average Epoch Loss')
    plt.suptitle('Average Loss')
    plt.plot(data_train)
    plt.plot(data_val)
    plt.legend(['Training', 'Validation'], loc='upper right', fancybox=True)
    plt.savefig("training_loss.png", dpi=300, bbox_inches='tight')

def model_eval(features, labels, model, dataset):
    with torch.no_grad():
        model.to(device)
        model.eval()
        predicted_classes = []
        actual_classes = []
    with torch.no_grad():
        for i, data in enumerate(features):
            inputs = torch.tensor(data).to(device)
            label = torch.tensor(labels[i]).to(device)
            output = model(inputs)
            prediction = output.argmax(dim=-1, keepdim=True)
            predicted_classes.append(prediction.item())
            actual_classes.append(label.item())

#         for i, data in enumerate(features):
#             inputs = torch.tensor(data).to(device)
#             labels = torch.tensor(labels[i]).to(device)
#             outputs = model(inputs)
#             predictions = outputs.argmax(dim=-1, keepdim=True)
#             predicted_classes.append(predictions.extend(predictions).item())
#             actual_classes.append(labels.item())

    #since labels are read in based on ordering in the folder, 
    #this corrects the labels so they reflect the correct classes
    #############################################################
    if dataset == 'gen':
        labels_dict = {
            0:3,
            1:5,
            2:7,
            3:8
        }
        for index, item in enumerate(actual_classes):
            actual_classes[index] = labels_dict[item]
    #############################################################

    performance_report = classification_report(
                        actual_classes, 
                        predicted_classes, 
                        labels=list(range(0,10)), 
                        target_names=classes, 
                        output_dict=True
                        )

    with open(f'performance_report_{dataset}.json', 'w') as f:
        json.dump(performance_report, f, indent=0)

    overall_accuracy = accuracy_score(actual_classes, predicted_classes)

    comparison_list = [['Actual', 'Predicted']]

    for i in range(0,len(actual_classes)):
        comparison_list.append([actual_classes[i], predicted_classes[i]])
    np.savetxt(f'class_pred_{dataset}.csv', comparison_list, delimiter=',', fmt='%s')

    labeled_actual = []
    labeled_predicted = []
    for index, item in enumerate(actual_classes):
        labeled_actual.append(classes[actual_classes[index]])
        labeled_predicted.append(classes[predicted_classes[index]])

    plt.clf()
    c_matrix = confusion_matrix(labeled_actual, labeled_predicted)
    c_df = pd.DataFrame(c_matrix, index=classes, columns=classes)
    plt.figure(figsize=(13,13))
    sns.heatmap(c_df, annot=True, fmt='g')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    plt.savefig(f'labeled_confusion_matrix_{dataset}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return overall_accuracy

class NeuralNet(torch.nn.Module):
    def __init__(self, config):
        super(NeuralNet, self).__init__()
        
        self.hl = config['h_layers']
        
        self.hidden_neurons = config['h_neurons']
        
        self.input = torch.nn.Linear(n_inputs, self.hidden_neurons)
        
        if config['h_layers']>=1:
            self.hidden_1 = torch.nn.Linear(self.hidden_neurons, self.hidden_neurons)
            
        if config['h_layers']==2:
            self.hidden_2 = torch.nn.Linear(self.hidden_neurons, self.hidden_neurons)
        
        self.output = torch.nn.Linear(self.hidden_neurons, 10)


    def forward(self, x):
        x = self.input(x)
        x = F.relu(x)
        
        if self.hl>=1:
            x = self.hidden_1(x)
            x = F.relu(x)
            
        if self.hl==2:
            x = self.hidden_2(x)
            x = F.relu(x)
        
        x = self.output(x)
        return x

In [9]:
def train_FFNN(config, checkpoint_dir=None):
    xtrain = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/train_extracted_features.pt',map_location=device)
    ytrain = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/train_extracted_labels.pt',map_location=device)
    xtest = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/val_extracted_features.pt',map_location=device)
    ytest = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/val_extracted_labels.pt',map_location=device)
    
#     For initial tuning:
    model = NeuralNet(config)
    model.to(device)
    
#     For learning curve of best model:
#     plt.clf()
#     plot_learning_curves(xtrain, ytrain, xtest, ytest, model)
#     plt.savefig('learning_curve.png')

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    loss_hist = []
    loss_val_hist = []
    for epoch in range(20):  # loop over the dataset multiple times
        loss_epoch = []
        loss_val_epoch = []
        for i, data in enumerate(xtrain):
            inputs = torch.tensor(data).to(device)
            labels = torch.tensor(ytrain[i]).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss_epoch.append(loss.item())
            loss.backward()
            optimizer.step()
        loss_hist.append(np.mean(loss_epoch))
        for i, data in enumerate(xtest):
            inputs = torch.tensor(data).to(device)
            labels = torch.tensor(ytest[i]).to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss_val_epoch.append(loss.item())
        loss_val_hist.append(np.mean(loss_val_epoch))
    
    loss_result = [['Training', 'Validation']]
    for i in range(0,len(loss_hist)):
        loss_result.append([loss_hist[i], loss_val_hist[i]])
    np.savetxt(f'loss.csv', loss_result, delimiter=',', fmt='%s')
    
#     np.savetxt('training_loss.csv', loss_hist, delimiter=',')
#     np.savetxt('training_val_loss.csv', loss_val_hist, delimiter=',')
    plot_loss(loss_hist, loss_val_hist)
    
    with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save((model.state_dict(), optimizer.state_dict()), path)
    
    train_acc =  model_eval(xtrain, ytrain, model, 'train')
    val_acc = model_eval(xtest, ytest, model, 'val')
    
    tune.report(
        train_ACC=train_acc,
        val_ACC=val_acc,
    )

In [10]:
# obtaining scale for hyperparameter tuning
xtrain = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/train_extracted_features.pt',map_location=device)
n_inputs = xtrain.shape[1]

In [14]:
def main(num_samples=15):
    config = {
        'h_layers':tune.choice([0, 1, 2]),
        'h_neurons':tune.choice([64, 128, 256]),
        'lr':tune.choice([1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    }
            
    algo=TuneBOHB(metric='train_ACC', 
                  mode='max'
                 )
    
    bohb = HyperBandForBOHB(time_attr="training_iteration",
                            metric="train_ACC",
                            mode="max",
                            max_t=1
                           )
        
    result = tune.run(
        tune.with_parameters(train_FFNN),
        resources_per_trial={"cpu": 0, "gpu": 1},
        config=config,
        num_samples=num_samples,
        scheduler=bohb,
        search_alg=algo,
        progress_reporter=tune.JupyterNotebookReporter(overwrite=True, print_intermediate_tables=True),
        fail_fast=True, 
        sync_config=tune.SyncConfig(
        syncer=None  # Disable syncing
        )
    )
    
    result.results_df.to_csv(f'results_df_{model_type}.csv')
    return result
# BOHB - https://arxiv.org/abs/1807.01774
# https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-scheduler-bohb

In [18]:
result_ffnn = main()

Trial name,status,loc,h_layers,h_neurons,lr,iter,total time (s),train_ACC,val_ACC
train_FFNN_3090b61c,TERMINATED,127.0.0.1:25400,2,256,0.0001,1,2035.05,1,0.999
train_FFNN_30ab1ceb,TERMINATED,127.0.0.1:16352,1,128,0.001,1,2214.47,0.999933,0.9986
train_FFNN_f08e50c1,TERMINATED,127.0.0.1:24244,0,64,1e-05,1,1204.43,0.999844,0.9994
train_FFNN_1ad3d0cd,TERMINATED,127.0.0.1:17064,1,128,1e-05,1,1320.08,0.999978,0.9992
train_FFNN_eb7c899c,TERMINATED,127.0.0.1:14532,0,256,0.001,1,1030.84,1,0.9992
train_FFNN_00eba969,TERMINATED,127.0.0.1:3920,1,128,0.0001,1,1380.65,1,0.9992
train_FFNN_69dda953,TERMINATED,127.0.0.1:17996,1,64,0.0001,1,1642.44,1,0.999
train_FFNN_a35ed4f0,TERMINATED,127.0.0.1:7496,0,64,0.0001,1,1277.45,1,0.9994
train_FFNN_78fd4315,TERMINATED,127.0.0.1:3216,0,256,0.0001,1,1016.06,1,0.9994
train_FFNN_74be7528,TERMINATED,127.0.0.1:24136,0,64,0.01,1,953.644,0.9996,0.9984


2022-07-23 08:18:21,970	INFO tune.py:639 -- Total run time: 22056.30 seconds (22056.16 seconds for the tuning loop).


 pid=18212) Figure(1300x1300)
 pid=18212) Figure(1300x1300)


BOHB Example: https://docs.ray.io/en/latest/tune/examples/includes/bohb_example.html